In [1]:
#import modules
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

#create Spark session
appName = "Sentiment Analysis in Spark"
conf = (SparkConf().setAppName("appName"))

sc = SparkContext.getOrCreate()
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.sql.streaming.schemaInference", "true") \
    .config("appName", "TweetStream") \
    .getOrCreate() 

23/01/19 19:34:11 WARN Utils: Your hostname, mecha resolves to a loopback address: 127.0.1.1; using 192.168.1.161 instead (on interface wlp8s0)
23/01/19 19:34:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/19 19:34:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/19 19:34:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
basePath = "/home/mecha/Documents/ml_models/sentiment_analyzer"
model = LogisticRegressionModel.load(basePath + "/modeliter11Demo")

In [3]:
columns = ["text", "label"]
# data = [("I am happy with Java", 4), ("Unhappy with Python", 0), ("Scala sucks as a technology", 0)]
predictionData = spark \
    .readStream \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv("tweetstream/*.csv",header=True) 
# rdd = spark.sparkContext.parallelize(data)
# predictionData = rdd.toDF(columns)
# predictionData.show()


tokenizer = Tokenizer(inputCol="text", outputCol="SentimentWords")
tokenizedTest = tokenizer.transform(predictionData)
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTest = swr.transform(tokenizedTest)
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTest = hashTF.transform(SwRemovedTest).select('text', 'MeaningfulWords', 'features')



In [4]:
prediction = model.transform(numericTest)
prediction = prediction.select("text", "prediction")
tweetStream = prediction.writeStream \
  .format("csv") \
  .trigger(processingTime="10 seconds") \
  .option("checkpointLocation", "checkpoint/") \
  .option("path", "predictions_out/") \
  .outputMode("append") \
  .start() \
  .awaitTermination()

23/01/19 19:34:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/01/19 19:37:40 WARN DAGScheduler: Broadcasting large task binary with size 10.2 MiB
23/01/19 19:38:10 WARN DAGScheduler: Broadcasting large task binary with size 10.2 MiB
23/01/19 19:38:40 WARN DAGScheduler: Broadcasting large task binary with size 10.2 MiB
23/01/19 19:39:21 WARN DAGScheduler: Broadcasting large task binary with size 10.2 MiB


23/01/19 19:40:00 WARN DAGScheduler: Broadcasting large task binary with size 10.2 MiB


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/mecha/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mecha/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 